# Algorithmic Trading Model Development for BTC/USDT Crypto Market

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

## Datasets

In [6]:
df_3m  = pd.read_csv('datasets/btc_3m.csv')
df_5m  = pd.read_csv('datasets/btc_5m.csv')
df_15m = pd.read_csv('datasets/btc_15m.csv')
df_30m = pd.read_csv('datasets/btc_30m.csv')
df_30m = pd.read_csv('datasets/btc_30m.csv')
df_1h  = pd.read_csv('datasets/btc_1h.csv')
df_2h  = pd.read_csv('datasets/btc_2h.csv')
df_4h  = pd.read_csv('datasets/btc_4h.csv')
df_6h  = pd.read_csv('datasets/btc_6h.csv')

Adding technical indicators (RSI, MACD-Signal Line, Moving averages, Bollinger bands, ATR)

In [ ]:
def add_indicators(df):
    # SMA
    df['SMA_14'] = df['close'].rolling(window=14).mean()
    df['SMA_20'] = df['close'].rolling(window=20).mean()

    # RSI
    delta = df['close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()

    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))

    # Moving Average Convergence Divergence (MACD)
    short_window = 12
    long_window = 26
    signal_window = 9

    exp_short = df['close'].ewm(span=short_window, adjust=False).mean()
    exp_long = df['close'].ewm(span=long_window, adjust=False).mean()

    df['MACD'] = exp_short - exp_long
    df['Signal_Line'] = df['MACD'].ewm(span=signal_window, adjust=False).mean()

    # Bollinger Bands
    window = 20
    df['Upper_Band'] = df['SMA_20'] + 2 * df['close'].rolling(window=window).std()
    df['Lower_Band'] = df['SMA_20'] - 2 * df['close'].rolling(window=window).std()

    # ATR
    # Calculate True Range (TR)
    df['High-Low'] = df['high'] - df['low']
    df['High-PrevClose'] = abs(df['high'] - df['close'].shift(1))
    df['Low-PrevClose'] = abs(df['low'] - df['close'].shift(1))

    df['TrueRange'] = df[['High-Low', 'High-PrevClose', 'Low-PrevClose']].max(axis=1)

    # Choose the ATR period (e.g., 14 days)
    atr_period = 14
    df['EMA'] = df['close'].ewm(span=15, adjust=False).mean()
    # Calculate ATR using rolling mean
    df['ATR'] = df['TrueRange'].rolling(window=atr_period).mean()

    short_window = 20  # Example: 20-minute moving average
    long_window = 50   # Example: 50-minute moving average
    df['short_ma'] = df['close'].rolling(window=short_window, min_periods=1).mean()

    # Calculate long-term moving average (long_ma)
    df['long_ma'] = df['close'].rolling(window=long_window, min_periods=1).mean()

    # Drop intermediate columns used for calculation
    df.drop(['High-Low', 'High-PrevClose', 'Low-PrevClose', 'TrueRange'], axis=1, inplace=True)

    # Removing null rows
    df = df.dropna()
    return df